In [1]:
# Packages needed
import pandas as pd
from dataprep.clean import clean_country

ModuleNotFoundError: No module named 'dataprep'

In [ ]:
# Paths to datasets
train_data = "datasets_hi4/train-data.csv"
GSPCI_data = "datasets_hi4/extra-dataset/GSCPI_data.csv"
LPI_data = "datasets_hi4/extra-dataset/LPIextend.csv"
country_code = "datasets_hi4/extra-dataset/country_code.csv"

In [ ]:
# Initializing datasets
td = pd.read_csv(train_data, sep=";")
GSPCId = pd.read_csv(GSPCI_data, sep=",")
LPId = pd.read_csv(LPI_data, sep=",")
cd = pd.read_csv(country_code, sep=",")

In [ ]:
# Convert last column to int
td = td.dropna(subset=["Month 1"])
for column in ["Month 1", "Month 2", "Month 3", "Month 4"]:
    td[column] = td[column].str.replace(" ", "").astype(float)

In [ ]:
td.drop_duplicates(inplace=True)

In [ ]:
# Merging LPI Mean score to train data
columns = []
for column in LPId.columns:
    if "Score" in column:
        columns.append(column)
LPId["LPI Mean Score"] = LPId[columns].mean(axis=1)
LPId = clean_country(LPId, "Country", output_format="alpha-2", inplace=True)
LPId = LPId[["Country (A2)", "LPI Mean Score"]].rename(columns={"Country (A2)": "Country"})
td = pd.merge(td, LPId, on = "Country")

In [ ]:
month_lib = {
    "jan": 0,
    "feb": 1,    
    "mar": 2,
    "apr": 3,    
    "may": 4,
    "jun": 5,    
    "jul": 6,
    "aug": 7,    
    "sep": 8,
    "oct": 9,    
    "nov": 10,
    "dec": 11
}

In [ ]:
date = td["Date"].str.split(n=1, expand=True)
td["Year"] = date[1]
td["Quarter"] = pd.to_numeric(date[0].str.slice(stop=3).replace(month_lib))//3
td["Quarter"] = td["Quarter"].astype(str)

In [ ]:
GSPCId["Year"] = GSCPId["Year-Month"].str.slice(stop=4)
GSCPId["Quarter"] = pd.to_numeric(GSCPId["Year-Month"].str.slice(start=5))//3
GSCPId["Quarter"] = GSCPId["Quarter"].astype(str)

In [ ]:
GSCPId = GSCPId.groupby(["Year", "Quarter"], as_index=False)["GSCPI"].mean()

In [ ]:
td = pd.merge(td, GSCPId, on=["Year", "Quarter"])

In [ ]:
td.to_csv("datesets_hi4/train_data_extra.csv",sep=";")